In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-07-04 14:33:31,134 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


In [ ]:
from agents import Agent, Runner


agent = Agent(
    name='Assistant',
    instructions='You are a helpful assistant',
    output_type=str,
)

result = await Runner.run(agent, 'Write a haiku about recursion in programming.')
print(result.final_output)

Code repeats itself,  
Functions within functions loop—  
Infinite echoes.


In [ ]:
TODO = ...

agent = Agent(
    name=TODO,
    instructions=TODO,
    prompt=TODO,
    handoff_description=TODO,
    handoffs=TODO,
    model=TODO,
    model_settings=TODO,
    tools=TODO,
    mcp_servers=TODO,
    mcp_config=TODO,
    input_guardrails=TODO,
    output_guardrails=TODO,
    output_type=TODO,
    hooks=TODO,
    tool_use_behavior=TODO,
    reset_tool_choice=TODO,
)

In [ ]:
# TODO use openai guardrail model

In [ ]:
from typing import Optional

import httpx

from agents import FunctionTool, function_tool
from agents.tool_context import ToolContext
from pydantic import BaseModel, Field


# Input model
class SandboxExecuteParams(BaseModel):
    user_id: str = Field(..., description="Identifier for the user's sandbox session")
    code: str = Field(..., description='Python code to run in the sandbox')
    timeout: Optional[float] = Field(
        None, description='Optional timeout in seconds (defaults to 30.0)'
    )

    def to_payload(self) -> dict:
        return self.model_dump(exclude_none=True)

    def to_json(self) -> str:
        return self.model_dump_json(exclude_none=True)


# Tool function
async def _invoke_sandbox_tool(tool_context: ToolContext, args_json_str: str) -> str:
    args = SandboxExecuteParams.model_validate_json(args_json_str)
    async with httpx.AsyncClient() as client:
        resp = await client.post(
            'https://your-sandbox.example.com/execute',
            json=args.to_payload(),
            timeout=args.timeout or 30.0,
        )
        resp.raise_for_status()
        body = resp.json()
        return body.get('output', '')


# Tool description
SANDBOX_EXECUTE_TOOL_DESCRIPTION = (
    'Execute Python code in the isolated sandbox. '
    'Parameters: user_id (string), code (string), optional timeout (float). '
    'Returns the stdout+stderr output.'
)

# Tool registration
sandbox_execute_tool = FunctionTool(
    name='sandbox_execute',
    description=SANDBOX_EXECUTE_TOOL_DESCRIPTION,
    params_json_schema=SandboxExecuteParams.model_json_schema(),
    on_invoke_tool=_invoke_sandbox_tool,
    strict_json_schema=True,
    is_enabled=True,
)

In [ ]:
from agents import Handoff
from agents.extensions.handoff_filters import remove_all_tools


tool: FunctionTool = ...
agent: Agent = ...

handoff = Handoff(
    tool_name=tool.name,
    tool_description=tool.description,
    input_json_schema=tool.params_json_schema,
    on_invoke_handoff=tool.on_invoke_tool,
    agent_name=agent.name,
    input_filter=remove_all_tools,
    strict_json_schema=True,
    is_enabled=True,
)

In [ ]:
from agents import HandoffInputData


def drop_pre_handoff_items(data: HandoffInputData) -> HandoffInputData:
    """
    Remove everything generated before the handoff turn,
    so the next agent only sees the original history + new_items.
    """
    return HandoffInputData(
        input_history=data.input_history,
        pre_handoff_items=(),
        new_items=data.new_items,
    )


def keep_last_5_new_items(data: HandoffInputData) -> HandoffInputData:
    """
    Keep only the last 5 items from new_items,
    preserving full history and pre_handoff_items.
    """
    return HandoffInputData(
        input_history=data.input_history,
        pre_handoff_items=data.pre_handoff_items,
        new_items=tuple(data.new_items[-5:]),
    )

In [ ]:
import openai


response = openai.moderations.create(
    model='omni-moderation-latest', input='This is a sample user message to classify.'
)

print(response)

In [3]:
openai_managed_mm = infrastructure_container.openai_managed_mm()

In [ ]:
await openai_managed_mm.moderate()

In [ ]:
async_openai_client = infrastructure_container.async_openai_client()

response = await async_openai_client.moderations.create(
    input='is this allowed input?', model='omni-moderation-latest'
)
response

ModerationCreateResponse(id='modr-9506', model='omni-moderation-latest', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=False, illicit_violent=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=False, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'], harassment/threatening=['text'], hate/threatening=['text'], i

In [ ]:
for result in response.results:
    if result.flagged:
        # NOTE: warn a user
        pass

    for moderation in result.categories:
        print(moderation)

    print('---')

('harassment', False)
('harassment_threatening', False)
('hate', False)
('hate_threatening', False)
('illicit', False)
('illicit_violent', False)
('self_harm', False)
('self_harm_instructions', False)
('self_harm_intent', False)
('sexual', False)
('sexual_minors', False)
('violence', False)
('violence_graphic', False)
('harassment/threatening', False)
('hate/threatening', False)
('illicit/violent', False)
('self-harm/intent', False)
('self-harm/instructions', False)
('self-harm', False)
('sexual/minors', False)
('violence/graphic', False)
---
